In [1]:
#import packages
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import qgrid
from urllib.request import urlretrieve

In [2]:
#declare url
url = "https://www.sec.gov/Archives/edgar/data/812011/0000927356-96-001229.txt"

In [3]:
#save file locally
urlretrieve(url, "MTN_10K")

('MTN_10K', <http.client.HTTPMessage at 0x10744e518>)

In [4]:
#open and read
TenK = open("MTN_10K", mode="r").readlines()

In [5]:
#locate start and end points of financial statements within TenK
a = "(IN THOUSANDS"
b = "The accompanying notes to consolidated financial statements"

start_points = []
end_points = []

for count, line in enumerate(TenK):
    if a in line:
        (start_points.append(count))
    if b in line:
        (end_points.append(count))

print(start_points)
print(end_points)

[1449, 1517, 1593, 1656, 2212]
[1508, 1584, 1647, 1740]


In [6]:
#zip start_points and end_points into a dictionary, all_points
all_points = (dict(zip(start_points, end_points)))
print(all_points)

{1449: 1508, 1517: 1584, 1593: 1647, 1656: 1740}


In [7]:
#slice MTN_10K by all_points and store resulting sub lists in a list, stmt_list

stmt_list = []

for key, value in all_points.items():
    x = key
    y = value
    stmt_list.append(TenK[x:y])

In [8]:
#join list of strings, stmt_list, into one long string
stmt1 = "".join(stmt_list[0])
stmt2 = "".join(stmt_list[1])
stmt3 = "".join(stmt_list[2])

statements = [stmt1, stmt2, stmt3]

In [55]:
#define delimeter locations

#stmt1
column_locator1 = len(str(re.search(r"[a-zA-Z0-9]", stmt1).expand))
column_locator2 = len(str(re.search(r"\.", stmt1)))
pattern1 = "(." + "{" + "{}".format(column_locator1) + "}"+ ")"
pattern2 = "(." + "{" + "{}".format(column_locator2) + "}"+ ")"

print(column_locator1)
print(column_locator2)

64
51


In [54]:
#add the delimters to each statement
stmt1 = re.sub(pattern1, r"\1|", stmt1)
stmt1 = re.sub(pattern2, r"\1|", stmt1)

In [13]:
#write the contents of each statement its own file
with open("TenK_stmt1", mode="w+") as TenK_stmt1:
    TenK_stmt1.write(stmt1)
    TenK_stmt1.close()

In [53]:
#read file into a pandas dataframe
df = pd.read_csv("TenK_stmt1", sep="|", header=None)
df = df.replace("\.|\-|=|<|>|,|\(\D\D\D\D.+|\d\)", "", regex = True).replace("TABLE", "").replace("CAPTION", "").replace("/TABLE","")